In [1]:
import pandas as pd

In [2]:
df_robusta = pd.read_csv('../data/raw/robusta.csv')

print("--- Primeiras linhas do Arquivo ---")
display(df_robusta.head())

print("\n--- Informações dos Tipos de Dados ---")
df_robusta.info()



--- Primeiras linhas do Arquivo ---


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,16.01.2026,"4.013,00","3.988,00","4.052,00","3.979,00",NaN,"2,29%"
1,15.01.2026,"3.923,00","3.864,00","3.953,00","3.803,00","5,25K","1,13%"
2,14.01.2026,"3.879,00","3.889,00","3.919,00","3.838,00","3,36K","-0,08%"
3,13.01.2026,"3.882,00","3.861,00","3.891,00","3.846,00","2,89K","0,96%"
4,12.01.2026,"3.845,00","3.800,00","3.862,00","3.797,00","2,95K","0,21%"



--- Informações dos Tipos de Dados ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Data      1526 non-null   object
 1   Último    1526 non-null   object
 2   Abertura  1526 non-null   object
 3   Máxima    1526 non-null   object
 4   Mínima    1526 non-null   object
 5   Vol.      1522 non-null   object
 6   Var%      1526 non-null   object
dtypes: object(7)
memory usage: 83.6+ KB


In [8]:
df_robusta = pd.read_csv('../data/raw/robusta.csv')

df_robusta['Date'] = pd.to_datetime(df_robusta['Data'], dayfirst=True)

df_robusta['Último'] = df_robusta['Último'].astype(str).str.replace('.', '', regex=False)

df_robusta['Último'] = df_robusta['Último'].str.replace(',', '.', regex=False)

df_robusta['Close_Robusta'] = df_robusta['Último'].astype(float)

df_robusta_limpo = df_robusta[['Date', 'Close_Robusta']].sort_values('Date')

# Verificação Final
df_robusta_limpo.info()
display(df_robusta_limpo.head())

<class 'pandas.core.frame.DataFrame'>
Index: 1526 entries, 1525 to 0
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           1526 non-null   datetime64[ns]
 1   Close_Robusta  1526 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 35.8 KB


,Date,Close_Robusta
1525,2020-01-02,1380.0
1524,2020-01-03,1372.0
1523,2020-01-06,1352.0
1522,2020-01-07,1364.0
1521,2020-01-08,1354.0


In [14]:
df_sistema = pd.read_parquet('../data/raw/dados_brutos_commodities.parquet')

df_sistema = df_sistema['Close']

df_sistema = df_sistema.reset_index()

try:
    df_sistema['Date'] = df_sistema['Date'].dt.tz_localize(None)
except:
    pass 

df_final = pd.merge(df_sistema, df_robusta_limpo, on='Date', how='outer')

df_final = df_final.sort_values('Date').set_index('Date')

df_final.columns = ['Dolar', 'Arabica', 'Robusta']

print("--- ✅ SUCESSO! Tabela Unificada ---")
display(df_final.tail(10))
df_final.info()

--- ✅ SUCESSO! Tabela Unificada ---


,Dolar,Arabica,Robusta
Date,,,
2026-01-05,5.4216,359.350006,3844.0
2026-01-06,5.4075,373.850006,3930.0
2026-01-07,5.3720,375.450012,3876.0
2026-01-08,5.3839,372.350006,3866.0
2026-01-09,5.3866,357.649994,3837.0
2026-01-12,5.3703,356.049988,3845.0
2026-01-13,5.3737,360.250000,3882.0
2026-01-14,5.3713,356.000000,3879.0
2026-01-15,5.3961,358.100006,3923.0


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1565 entries, 2020-01-02 to 2026-01-16
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Dolar    1300 non-null   float64
 1   Arabica  1259 non-null   float64
 2   Robusta  1526 non-null   float64
dtypes: float64(3)
memory usage: 48.9 KB


In [17]:
df_final.columns = ['Dolar', 'Arabica', 'Robusta']

# Salvando
df_final.to_parquet('../data/processed/dados_completos_commodities.parquet')
print("✅ Dados Salvos!")

✅ Dados Salvos!
